In [ ]:
!pip install langchain langchain-core langchain-community pypdf pinecone pinecone-text langchain_openai openai

In [ ]:
import os
from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
from pinecone import Pinecone, ServerlessSpec
from pinecone_text.sparse import BM25Encoder
from openai import OpenAI
import json

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving hr.zip to hr.zip


In [ ]:
!unzip hr.zip -d hr_rag

Archive:  hr.zip
   creating: hr_rag/hr/COMPANY-SPECIFIC DOCUMENT/
  inflating: hr_rag/hr/COMPANY-SPECIFIC DOCUMENT/Mission, Vision.pdf  
  inflating: hr_rag/hr/COMPANY-SPECIFIC DOCUMENT/Security.pdf  
   creating: hr_rag/hr/EMPLOYEE_LIFECYCLE_DOCUMENT/
  inflating: hr_rag/hr/EMPLOYEE_LIFECYCLE_DOCUMENT/Appraisal.pdf  
  inflating: hr_rag/hr/EMPLOYEE_LIFECYCLE_DOCUMENT/Employee Onboarding Checklist.pdf  
  inflating: hr_rag/hr/EMPLOYEE_LIFECYCLE_DOCUMENT/IT Onboarding.pdf  
  inflating: hr_rag/hr/EMPLOYEE_LIFECYCLE_DOCUMENT/Performance Improvement Plan.pdf  
  inflating: hr_rag/hr/EMPLOYEE_LIFECYCLE_DOCUMENT/Training.pdf  
   creating: hr_rag/hr/hr_policy_documents/
  inflating: hr_rag/hr/hr_policy_documents/Code of Conduct.pdf  
  inflating: hr_rag/hr/hr_policy_documents/Disciplinary Action Policy.pdf  
  inflating: hr_rag/hr/hr_policy_documents/Employee Exit.pdf  
  inflating: hr_rag/hr/hr_policy_documents/Leave_and_Attendance_Policy.pdf  
  inflating: hr_rag/hr/hr_policy_documents/p

In [ ]:
import os
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import PyPDFLoader

root_dir = "/content/hr_rag"


loader = DirectoryLoader(
    root_dir,
    glob="**/*.pdf",
    loader_cls=PyPDFLoader,
    recursive=True
)

documents = loader.load()

print("Total documents loaded:", len(documents))
documents[:2]

Total documents loaded: 138


[Document(metadata={'producer': 'www.ilovepdf.com', 'creator': 'Microsoft® Word 2016', 'creationdate': '2025-11-21T06:22:38+00:00', 'author': 'Tanish Sharma', 'moddate': '2025-11-21T06:22:38+00:00', 'source': '/content/hr_rag/hr/PAYROLL & BENEFITS DOCUMENT/Employee Benefits.pdf', 'total_pages': 7, 'page': 0, 'page_label': '1'}, page_content='Employee Benefits & Allowances Policy \n1. Purpose \nThis document outlines the various benefits, allowances, and financial support provided to \nemployees as part of their overall compensation package. These benefits are designed to enhance \nemployee well-being, support productivity, and promote work-life balance. \nThis policy applies to all full-time employees unless otherwise specified. \n \n2. Food Allowance \n2.1 Monthly Food Allowance \nEmployees may be eligible for a monthly food allowance that can be used for: \n\uf0b7 Lunch/meals during office hours \n\uf0b7 Cafeteria usage \n\uf0b7 Food service partners (if available) \n2.2 Allowance Am

In [ ]:
for doc in documents:
    print(doc.metadata["source"], doc.metadata["page"])

/content/hr_rag/hr/PAYROLL & BENEFITS DOCUMENT/Employee Benefits.pdf 0
/content/hr_rag/hr/PAYROLL & BENEFITS DOCUMENT/Employee Benefits.pdf 1
/content/hr_rag/hr/PAYROLL & BENEFITS DOCUMENT/Employee Benefits.pdf 2
/content/hr_rag/hr/PAYROLL & BENEFITS DOCUMENT/Employee Benefits.pdf 3
/content/hr_rag/hr/PAYROLL & BENEFITS DOCUMENT/Employee Benefits.pdf 4
/content/hr_rag/hr/PAYROLL & BENEFITS DOCUMENT/Employee Benefits.pdf 5
/content/hr_rag/hr/PAYROLL & BENEFITS DOCUMENT/Employee Benefits.pdf 6
/content/hr_rag/hr/PAYROLL & BENEFITS DOCUMENT/General Reimbursement Rules Document.pdf 0
/content/hr_rag/hr/PAYROLL & BENEFITS DOCUMENT/General Reimbursement Rules Document.pdf 1
/content/hr_rag/hr/PAYROLL & BENEFITS DOCUMENT/General Reimbursement Rules Document.pdf 2
/content/hr_rag/hr/PAYROLL & BENEFITS DOCUMENT/General Reimbursement Rules Document.pdf 3
/content/hr_rag/hr/PAYROLL & BENEFITS DOCUMENT/General Reimbursement Rules Document.pdf 4
/content/hr_rag/hr/PAYROLL & BENEFITS DOCUMENT/Genera

In [ ]:
from collections import defaultdict
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
docs_by_source = defaultdict(list)
for page in documents:
    docs_by_source[page.metadata['source']].append(page)

merged_docs = []
for source, page_list in docs_by_source.items():
    page_list_sorted = sorted(page_list, key=lambda x: x.metadata.get('page', 0))

    full_text = "\n\n".join([p.page_content for p in page_list_sorted])

    merged_docs.append(
        Document(
            page_content=full_text,
            metadata={"source": source}
        )
    )

In [ ]:
merged_docs[0]

Document(metadata={'source': '/content/hr_rag/hr/PAYROLL & BENEFITS DOCUMENT/Employee Benefits.pdf'}, page_content='Employee Benefits & Allowances Policy \n1. Purpose \nThis document outlines the various benefits, allowances, and financial support provided to \nemployees as part of their overall compensation package. These benefits are designed to enhance \nemployee well-being, support productivity, and promote work-life balance. \nThis policy applies to all full-time employees unless otherwise specified. \n \n2. Food Allowance \n2.1 Monthly Food Allowance \nEmployees may be eligible for a monthly food allowance that can be used for: \n\uf0b7 Lunch/meals during office hours \n\uf0b7 Cafeteria usage \n\uf0b7 Food service partners (if available) \n2.2 Allowance Amount \n\uf0b7 Junior staff: ₹1,000–₹2,000 per month \n\uf0b7 Mid-level: ₹1,500–₹2,500 per month \n\uf0b7 Senior staff: ₹2,000–₹3,000 per month \n(Exact values vary by company policy.) \n2.3 Meal Card / Wallet \nCompanies may iss

In [ ]:
import os
from collections import defaultdict
from langchain_core.documents import Document

docs_by_source = defaultdict(list)

for page in documents:
    docs_by_source[page.metadata["source"]].append(page)

merged_docs = []

for raw_source, page_list in docs_by_source.items():
    page_list_sorted = sorted(page_list, key=lambda x: x.metadata.get("page", 0))

    full_text = "\n\n".join([p.page_content for p in page_list_sorted])

    folder_name = os.path.basename(os.path.dirname(raw_source))
    filename = os.path.basename(raw_source)
    file_stem, _ = os.path.splitext(filename)

    display_source = f"{folder_name}/{file_stem}"

    category = "general"

    if "COMPANY-SPECIFIC" in folder_name:
        category = "company_info"
    elif "EMPLOYEE_LIFECYCLE" in folder_name:
        category = "employee_lifecycle"
    elif "hr_policy" in folder_name:
        category = "hr_policy"
    elif "PAYROLL" in folder_name or "BENEFITS" in folder_name:
        category = "payroll_benefits"

    employee = False
    manager = False
    hr = False

    if category == "company_info":
        hr = True
    elif category == "employee_lifecycle":
        manager = True
    elif category == "hr_policy":
        employee = True
    elif category == "payroll_benefits":
        employee = True
        manager = True

    metadata = {
        "source": display_source,
        "category": category,
        "employee": employee,
        "manager": manager,
        "hr": hr,
    }

    merged_docs.append(
        Document(page_content=full_text, metadata=metadata)
    )

In [ ]:
len(merged_docs)

20

In [ ]:
print(merged_docs[0])

page_content='Employee Benefits & Allowances Policy 
1. Purpose 
This document outlines the various benefits, allowances, and financial support provided to 
employees as part of their overall compensation package. These benefits are designed to enhance 
employee well-being, support productivity, and promote work-life balance. 
This policy applies to all full-time employees unless otherwise specified. 
 
2. Food Allowance 
2.1 Monthly Food Allowance 
Employees may be eligible for a monthly food allowance that can be used for: 
 Lunch/meals during office hours 
 Cafeteria usage 
 Food service partners (if available) 
2.2 Allowance Amount 
 Junior staff: ₹1,000–₹2,000 per month 
 Mid-level: ₹1,500–₹2,500 per month 
 Senior staff: ₹2,000–₹3,000 per month 
(Exact values vary by company policy.) 
2.3 Meal Card / Wallet 
Companies may issue: 
 Prepaid meal cards 
 Digital food wallets 
 Sodexo/Swiggy Money vouchers 
2.4 Tax Benefit 
Meal allowance through Meal Card may be tax-exempt 

In [ ]:
parent_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
    separators=["\n\n", "\n", ". ", " ", ""]
)

import os
import re

parent_chunks = []

for doc in merged_docs:
    splits = parent_splitter.split_text(doc.page_content)

    clean_base = doc.metadata["source"].replace(" ", "_").replace("/", "_")

    for i, text in enumerate(splits):

        parent_meta = doc.metadata.copy()

        parent_meta["parent_id"] = f"{clean_base}_parent_{i}"

        parent_chunks.append(
            Document(
                page_content=text,
                metadata=parent_meta
            )
        )

In [ ]:
print(parent_chunks[0])

page_content='Employee Benefits & Allowances Policy 
1. Purpose 
This document outlines the various benefits, allowances, and financial support provided to 
employees as part of their overall compensation package. These benefits are designed to enhance 
employee well-being, support productivity, and promote work-life balance. 
This policy applies to all full-time employees unless otherwise specified. 
 
2. Food Allowance 
2.1 Monthly Food Allowance 
Employees may be eligible for a monthly food allowance that can be used for: 
 Lunch/meals during office hours 
 Cafeteria usage 
 Food service partners (if available) 
2.2 Allowance Amount 
 Junior staff: ₹1,000–₹2,000 per month 
 Mid-level: ₹1,500–₹2,500 per month 
 Senior staff: ₹2,000–₹3,000 per month 
(Exact values vary by company policy.) 
2.3 Meal Card / Wallet 
Companies may issue: 
 Prepaid meal cards 
 Digital food wallets 
 Sodexo/Swiggy Money vouchers 
2.4 Tax Benefit 
Meal allowance through Meal Card may be tax-exempt 

In [ ]:
import json

output_path = "parent_chunks.jsonl"

with open(output_path, "w", encoding="utf-8") as f:
    for parent in parent_chunks:
        obj = {
            "id": parent.metadata["parent_id"],
            "text": parent.page_content,
            "metadata": parent.metadata
        }
        f.write(json.dumps(obj, ensure_ascii=False) + "\n")

print(f"Saved {len(parent_chunks)} parent chunks to {output_path}")

Saved 120 parent chunks to parent_chunks.jsonl


In [ ]:
loaded_parents = []
with open("parent_chunks.jsonl", "r", encoding="utf-8") as f:
    for line in f:
        loaded_parents.append(json.loads(line))

In [ ]:
loaded_parents[0]

{'id': 'PAYROLL_&_BENEFITS_DOCUMENT_Employee_Benefits_parent_0',
 'text': 'Employee Benefits & Allowances Policy \n1. Purpose \nThis document outlines the various benefits, allowances, and financial support provided to \nemployees as part of their overall compensation package. These benefits are designed to enhance \nemployee well-being, support productivity, and promote work-life balance. \nThis policy applies to all full-time employees unless otherwise specified. \n \n2. Food Allowance \n2.1 Monthly Food Allowance \nEmployees may be eligible for a monthly food allowance that can be used for: \n\uf0b7 Lunch/meals during office hours \n\uf0b7 Cafeteria usage \n\uf0b7 Food service partners (if available) \n2.2 Allowance Amount \n\uf0b7 Junior staff: ₹1,000–₹2,000 per month \n\uf0b7 Mid-level: ₹1,500–₹2,500 per month \n\uf0b7 Senior staff: ₹2,000–₹3,000 per month \n(Exact values vary by company policy.) \n2.3 Meal Card / Wallet \nCompanies may issue: \n\uf0b7 Prepaid meal cards \n\uf0b7 

In [ ]:
child_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=80,
    separators=["\n\n", "\n", ". ", " ", ""]
)

child_chunks = []

for parent in parent_chunks:
    splits = child_splitter.split_text(parent.page_content)

    for j, text in enumerate(splits):

        child_meta = parent.metadata.copy()
        child_meta["child_id"] = f"{parent.metadata['parent_id']}_child_{j}"

        child_chunks.append(
            Document(
                page_content=text,
                metadata=child_meta
            )
        )

In [ ]:
print(child_chunks[0])

page_content='Employee Benefits & Allowances Policy 
1. Purpose 
This document outlines the various benefits, allowances, and financial support provided to 
employees as part of their overall compensation package. These benefits are designed to enhance 
employee well-being, support productivity, and promote work-life balance. 
This policy applies to all full-time employees unless otherwise specified. 
 
2. Food Allowance 
2.1 Monthly Food Allowance' metadata={'source': 'PAYROLL & BENEFITS DOCUMENT/Employee Benefits', 'category': 'payroll_benefits', 'employee': True, 'manager': True, 'hr': False, 'parent_id': 'PAYROLL_&_BENEFITS_DOCUMENT_Employee_Benefits_parent_0', 'child_id': 'PAYROLL_&_BENEFITS_DOCUMENT_Employee_Benefits_parent_0_child_0'}


In [ ]:
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings(openai_api_key="sk-proj-....")

In [ ]:
from openai import OpenAI
os.environ["OPENAI_API_KEY"] = "sk-proj-....."
client = OpenAI()

In [ ]:
os.environ["PINECONE_API_KEY"] = "pcsk_4Hg4FE_BEkGnr6kma3vfRXdp1pvZfocyfTcmiJz3EoSturojEEHf7ubG9JvoL5tGNfeVz1"

In [ ]:
embedded_child_chunks = []

for doc in child_chunks:

    resp = client.embeddings.create(
        model="text-embedding-3-small",
        input=doc.page_content
    )

    vec = resp.data[0].embedding

    embedded_child_chunks.append({
        "id": doc.metadata["child_id"],
        "values": vec,
        "metadata": doc.metadata,
        "text": doc.page_content
    })

In [ ]:
embedded_child_chunks[0]

{'id': 'PAYROLL_&_BENEFITS_DOCUMENT_Employee_Benefits_parent_0_child_0',
 'values': [-0.009282061830163002,
  -0.012757004238665104,
  0.07071158289909363,
  0.054572924971580505,
  -0.05032836273312569,
  0.021829169243574142,
  0.01229056902229786,
  0.010121644474565983,
  0.03437627851963043,
  0.02749635837972164,
  0.03425966948270798,
  -0.034935999661684036,
  0.015287415124475956,
  -0.05821111798286438,
  -0.007427981123328209,
  0.016406860202550888,
  0.009999205358326435,
  -0.005527257453650236,
  0.02320515364408493,
  0.036591846495866776,
  0.005818779580295086,
  0.08255904167890549,
  -0.03593883663415909,
  0.045570723712444305,
  0.020966263487935066,
  -0.010028357617557049,
  -0.07971378415822983,
  -0.016383538022637367,
  -0.029781891033053398,
  -0.07444306463003159,
  0.02388148382306099,
  -0.012675377540290356,
  0.005154109559953213,
  0.004894654732197523,
  0.007952720858156681,
  0.019776854664087296,
  0.0024706493131816387,
  0.01715315692126751,
  0.

In [ ]:
from pinecone_text.sparse import BM25Encoder

bm25 = BM25Encoder.default()
bm25.fit([doc.page_content for doc in child_chunks])
sparse_vectors = bm25.encode_documents([doc.page_content for doc in child_chunks])

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


  0%|          | 0/339 [00:00<?, ?it/s]

In [ ]:
from pinecone import Pinecone, ServerlessSpec
import uuid
import os

pc = Pinecone()
index_name = "multi-rag-system"

if index_name not in pc.list_indexes().names():
  pc.create_index(
    name="multi-rag-system",
    dimension=1536,
    metric="dotproduct",
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    )
  )
index = pc.Index(index_name)
print("Index created:", index_name)

Index created: multi-rag-system


In [ ]:
for embedded, sparse in zip(embedded_child_chunks, sparse_vectors):

    metadata = embedded["metadata"].copy()
    metadata["text"] = embedded["text"]

    index.upsert([
        {
            "id": embedded["id"],
            "values": embedded["values"],
            "sparse_values": sparse,
            "metadata": metadata
        }
    ])